In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import requests
import pandas as pd
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification


In [3]:
# Step 1: Download the file
file_id = "1qpzy8eeqlSzkSN4g4yVLAa_ce0ZUovUh"
download_url = f"https://drive.google.com/uc?id={file_id}"
file_path = "dataset.conll"

response = requests.get(download_url)
with open(file_path, "wb") as file:
    file.write(response.content)

print(f"File downloaded and saved as {file_path}")

File downloaded and saved as dataset.conll


In [4]:
# Step 2: Load .conll data
def load_conll_data(file_path):
    sentences, labels, sentence, label = [], [], [], []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                token, entity = line.split()
                sentence.append(token)
                label.append(entity)
            else:
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                sentence, label = [], []
        if sentence:
            sentences.append(sentence)
            labels.append(label)
    return sentences, labels

sentences, labels = load_conll_data(file_path)
unique_labels = list(set(label for sublist in labels for label in sublist))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}
numerical_labels = [[label2id[label] for label in sublist] for sublist in labels]

data = {"tokens": sentences, "ner_tags": numerical_labels}
dataset = DatasetDict({"train": Dataset.from_dict(data)})

In [5]:
# Step 3: Tokenize and align labels
model_name = "rasyosef/bert-tiny-amharic"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_idx is None else label[word_idx] for word_idx in word_ids]
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/274k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/725k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Map:   0%|          | 0/1340 [00:00<?, ? examples/s]

In [6]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np

def compute_metrics(p):
    preds, labels = p
    # Get predictions by choosing the class with the highest probability
    preds = np.argmax(preds, axis=2)

    # Remove ignored index (usually -100) from the labels and predictions
    true_labels = [[label for label in label_seq if label != -100] for label_seq in labels]
    true_preds = [[pred for pred, label in zip(pred_seq, label_seq) if label != -100]
                  for pred_seq, label_seq in zip(preds, labels)]

    # Flatten the lists
    true_labels_flat = [item for sublist in true_labels for item in sublist]
    true_preds_flat = [item for sublist in true_preds for item in sublist]

    # Compute precision, recall, f1, and accuracy with zero_division to avoid ill-defined precision
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels_flat, true_preds_flat, average='macro', zero_division=1)

    accuracy = accuracy_score(true_labels_flat, true_preds_flat)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [7]:
# Step 4: Load model and set up trainer
num_labels = len(unique_labels)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

data_collator = DataCollatorForTokenClassification(tokenizer)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
)

train_size = int(0.8 * len(tokenized_datasets["train"]))
train_dataset = tokenized_datasets["train"].select(range(train_size))
eval_dataset = tokenized_datasets["train"].select(range(train_size, len(tokenized_datasets["train"])))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at rasyosef/bert-tiny-amharic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
# Step 5: Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.417800,0.944997,0.837477,0.150577,0.239443,0.160561
2,0.790400,0.577287,0.899792,0.170588,0.239090,0.190144
3,0.499400,0.464583,0.916548,0.192333,0.271043,0.217690
4,0.403000,0.411979,0.919834,0.197952,0.276585,0.224146
5,0.307700,0.384246,0.919286,0.245051,0.279902,0.225853
6,0.254700,0.368535,0.920710,0.294838,0.282785,0.233142
7,0.230500,0.360803,0.920819,0.289304,0.284527,0.235327
8,0.214700,0.353845,0.921805,0.301741,0.286455,0.240257
9,0.209500,0.349751,0.922352,0.298365,0.287427,0.243085
10,0.209600,0.348941,0.922462,0.298571,0.287445,0.243277


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

TrainOutput(global_step=1340, training_loss=0.4779829445169933, metrics={'train_runtime': 1135.6561, 'train_samples_per_second': 9.439, 'train_steps_per_second': 1.18, 'total_flos': 13097091563520.0, 'train_loss': 0.4779829445169933, 'epoch': 10.0})

In [9]:
results = trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
# Print results in a structured format
print("Evaluation Results:")
print(f"Loss: {results['eval_loss']:.4f}")
print(f"Accuracy: {results['eval_accuracy']:.4f}")
print(f"Precision: {results['eval_precision']:.4f}")
print(f"Recall: {results['eval_recall']:.4f}")
print(f"F1-Score: {results['eval_f1']:.4f}")
print(f"Runtime: {results['eval_runtime']:.4f} seconds")
print(f"Samples per second: {results['eval_samples_per_second']:.2f}")
print(f"Steps per second: {results['eval_steps_per_second']:.2f}")
print(f"Epoch: {results['epoch']:.1f}")

Evaluation Results:
Loss: 0.3489
Accuracy: 0.9225
Precision: 0.2986
Recall: 0.2874
F1-Score: 0.2433
Runtime: 6.8560 seconds
Samples per second: 39.09
Steps per second: 4.96
Epoch: 10.0


In [ ]:
# Save the model and tokenizer
model_save_path = "./saved_model_berta"  # Specify a directory to save the model
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to ./saved_model_berta
